In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#Helper functions for data processing
def card_to_num(card):
    raw_rank = card[:-1]
    
    ranks = {
        '2' : 0,
        '3' : 1,
        '4' : 2, 
        '5' : 3,
        '6' : 4, 
        '7' : 5, 
        '8' : 6, 
        '9' : 7, 
        '10': 8, 
        'J' : 9, 
        'Q' : 10, 
        'K' : 11, 
        'A': 12
    }

    return ranks[raw_rank]

def hand_to_list(hand):
    '''Takes hand like KH-AC and outputs list of card numbers'''
    hand_list_1 = hand.split("-")
    hand_list_2 = [card_to_num(card) for card in hand_list_1]
    return hand_list_2

result_mapping = {
    'hit' : 0,
    'stand' : 1,
    'double down' : 2
}

batch_size = 32

# Defining Dataset Class
class Blackjack_Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [3]:
# Data Processing
hit_stand_dd_df = pd.read_csv('CSVs/hit_stand_dd.csv')

In [4]:
# Cleaned hit_stand_dd
MAX_LEN = 7
hit_stand_dd_df['dealer_upcard'] = hit_stand_dd_df['dealer_upcard'].apply(card_to_num)
hit_stand_dd_df['player_hand'] = hit_stand_dd_df['player_hand'].apply(hand_to_list)
hit_stand_dd_df['result'] = hit_stand_dd_df['result'].map(result_mapping)
hit_stand_dd_df['player_hand'] = [
    hand + [0] * (MAX_LEN - len(hand)) if len(hand) < MAX_LEN else hand[:MAX_LEN] for hand in hit_stand_dd_df['player_hand']
]


# Turning into tensor matrices
# hit_stand_dd
x1 = torch.tensor(hit_stand_dd_df['player_hand'].to_list(), dtype=torch.float32)
x2 = torch.tensor(hit_stand_dd_df['dealer_upcard'].values, dtype=torch.float32).unsqueeze(1)
x3 = torch.tensor(hit_stand_dd_df['can_double'].values, dtype=torch.float32).unsqueeze(1)
y = torch.tensor(hit_stand_dd_df['result'].values, dtype=torch.long)

X = torch.cat([x1,x2,x3], dim=1)

hit_stand_dd_dataset = Blackjack_Dataset(X,y)
train_hsd, test_hsd = train_test_split(hit_stand_dd_dataset, test_size=0.2)

hsd_train_dataloader = DataLoader(train_hsd, batch_size=batch_size, shuffle=True)
hsd_test_dataloader = DataLoader(test_hsd, batch_size=batch_size, shuffle=True)

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    for batch, (X,y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Printing Training Update on every 100th batch
        if (batch + 1) % 100 == 0: 
            loss = loss.item()
            current = batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    #Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True

    with torch.no_grad():
        for X, y in dataloader: 
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size 
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
class hsd_NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(9, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

hsd_model = hsd_NeuralNetwork()

learning_rate = 0.0005 
epochs = 40

loss_fn = nn.CrossEntropyLoss()
hsd_optimizer = torch.optim.SGD(hsd_model.parameters(), lr=learning_rate)

In [9]:
for t in range(epochs):
    print(f"Epoch {t+1}\n---------------------------")
    train_loop(hsd_train_dataloader, hsd_model, loss_fn, hsd_optimizer)
    test_loop(hsd_test_dataloader, hsd_model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss: 0.970361  [ 3200/209452]
loss: 0.867522  [ 6400/209452]
loss: 0.894082  [ 9600/209452]
loss: 0.726908  [12800/209452]
loss: 0.900585  [16000/209452]
loss: 0.751150  [19200/209452]
loss: 0.729588  [22400/209452]
loss: 0.797868  [25600/209452]
loss: 0.697163  [28800/209452]
loss: 0.939191  [32000/209452]
loss: 0.846400  [35200/209452]
loss: 0.874388  [38400/209452]
loss: 0.601201  [41600/209452]
loss: 0.797319  [44800/209452]
loss: 0.665578  [48000/209452]
loss: 0.650254  [51200/209452]
loss: 0.704794  [54400/209452]
loss: 0.537581  [57600/209452]
loss: 0.709764  [60800/209452]
loss: 0.664131  [64000/209452]
loss: 0.654036  [67200/209452]
loss: 0.504442  [70400/209452]
loss: 0.672322  [73600/209452]
loss: 0.782632  [76800/209452]
loss: 0.698408  [80000/209452]
loss: 0.832971  [83200/209452]
loss: 0.465097  [86400/209452]
loss: 0.739153  [89600/209452]
loss: 0.813823  [92800/209452]
loss: 0.682154  [96000/209452]
loss: 0.641666  [99200/209452]
los